In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import cv2

import re
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
import shutil
import zipfile
import time
from urllib.request import urlretrieve
from tqdm import tqdm


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    dynamic_graph = tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    #graph = tf.get_default_graph()
    w1_out = sess.graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_out = sess.graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    w3_out = sess.graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    w4_out = sess.graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    w7_out = sess.graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return w1_out, keep_out, w3_out, w4_out, w7_out
tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    layer_1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, strides=(1,1), padding='same',
                               kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer_2 = tf.layers.conv2d_transpose(layer_1, num_classes, 4, strides=(2,2), padding='same',
                                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer_3_skip = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, strides=(1,1), padding='same',
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer_4_add = tf.add(layer_2, layer_3_skip)
    
    layer_5 = tf.layers.conv2d_transpose(layer_4_add, num_classes, 4, strides=(2,2), padding='same',
                                         kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                                         kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer_6_skip = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, strides=(1,1), padding='same',
                                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                                    kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    layer_7_add = tf.add(layer_5, layer_6_skip)
    
    last_layer = tf.layers.conv2d_transpose(layer_7_add, num_classes, 16, strides=(8,8), padding='same',
                                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.01), 
                                            kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))   
    
    
    
    return last_layer
#tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1, num_classes))
    adam_opt = tf.train.AdamOptimizer(learning_rate)
    
    with tf.name_scope('loss'):
        cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        reg_constant = 0.01
        cross_entropy_loss += reg_constant*sum(reg_losses)
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(correct_label, 1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        variable_summaries(correct_prediction)
    train_op = adam_opt.minimize(cross_entropy_loss)
        
    
    
    return logits, train_op, cross_entropy_loss
#tests.test_optimize(optimize)


def train_nn(sess, epochs, batch_size, get_batches_fn,get_batches_fn_test, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    step = 1
    for epoch in range(epochs):
        batch = 1
        test_batches = get_batches_fn_test(batch_size)
        for image, label in get_batches_fn(batch_size):
            summary, loss = sess.run([cross_entropy_loss, train_op], feed_dict={
                input_image: image,
                correct_label: label,
                keep_prob: 0.5,
            })
            #train_writer.add_summary(summary, step)
            print("step{}: epoch {}, batch {}, batch-loss: {}".format(step, epoch+1, batch, loss))
            if False and batch % 10 == 0:
                try:
                    test_image, test_label = next(text_batches)
                except StopIteration:
                    test_batches = get_batches_fn_test(batch_size)
                    test_image, test_label = next(test_batches)
                        
                    summary, loss = sess.run([cross_entropy_loss], feed_dict={
                        input_image: test_image,
                        correct_label: test_label,
                        keep_prob: 1.0,
                    })
                #test_writer.add_summary(summary, step)
                print("  test-batch-loss: {}".format(loss))
            batch +=1
            step += 1
                        
            
    

#tests.test_train_nn(train_nn)


def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    learning_rate = 0.001
    epochs = 9
    labels_tensor = tf.placeholder(tf.float32, [None, None, None, num_classes])
    batch_size = 20
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        get_batches_fn_test = helper.gen_batch_function(os.path.join(data_dir, 'data_road/testing'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        w1_out, keep_out, w3_out, w4_out, w7_out = load_vgg(sess, vgg_path)
        layer_output = layers(w3_out, w4_out, w7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(layer_output, labels_tensor, learning_rate, num_classes)
        
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
        
        timeStart = time.time()
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, get_batches_fn_test,
                train_op, cross_entropy_loss, w1_out, labels_tensor, keep_out,
                learning_rate)
        timeEnd = time.time()
        
        timeDif = timeEnd-timeStart
        
        timeStr = "Time spent Training is: " + str(timeDif)
        print(timeStr)
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_out, w1_out)
        data_dir = './Video'
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_out, w1_out)

        # OPTIONAL: Apply the trained model to a video
        #def gen_test_output(image_pl, sess, logits, keep_prob, data_folder, image_shape):

        
        # Import everything needed to edit/save/watch video clips
        from moviepy.editor import VideoFileClip
        from IPython.display import HTML
        #img = tf.placeholder
        '''
        def imgs4vid(img,sess=sess,logits=logits,keep_prob=keep_out):
            plt.imshow(img)
            plt.show()
            img = np.array(img, dtype='f')
            images = cv2.resize(img,(576,160))
            plt.imshow(images)
            plt.show()
            images = np.array(images, dtype='f')
            im_softmax = sess.run([tf.nn.softmax(logits)], {keep_prob: 1.0, img: [images]})
            im_softmax = im_softmax[0][:, 1].reshape(160, 576)
            segmentation = (im_softmax > 0.5).reshape(160, 576, 1)
            mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
            mask = scipy.misc.toimage(mask, mode="RGBA")
            street_im = scipy.misc.toimage(image)
            img = street_im.paste(mask, box=None, mask=mask)
            return img
        
        white_output = 'ChallengeVideoResult.mp4'
        clip1 = VideoFileClip("challenge_video.mp4")
        white_clip = clip1.fl_image(imgs4vid) #NOTE: this function expects color images!!
        %time white_clip.write_videofile(white_output, audio=False)
        '''

if __name__ == '__main__':
    run()

TensorFlow Version: 1.4.0


C:\Users\rvm\AppData\Local\Continuum\miniconda3\envs\carnd-term1\lib\site-packages\ipykernel_launcher.py:28: UserWarning: No GPU found. Please use a GPU to train your neural network.


Tests Passed
Tests Passed
INFO:tensorflow:Restoring parameters from b'./data\\vgg\\variables\\variables'
step1: epoch 1, batch 1, batch-loss: None
step2: epoch 1, batch 2, batch-loss: None
step3: epoch 1, batch 3, batch-loss: None
step4: epoch 1, batch 4, batch-loss: None
step5: epoch 1, batch 5, batch-loss: None
step6: epoch 1, batch 6, batch-loss: None
step7: epoch 1, batch 7, batch-loss: None
step8: epoch 1, batch 8, batch-loss: None
step9: epoch 1, batch 9, batch-loss: None
step10: epoch 1, batch 10, batch-loss: None
step11: epoch 1, batch 11, batch-loss: None
step12: epoch 1, batch 12, batch-loss: None
step13: epoch 1, batch 13, batch-loss: None
step14: epoch 1, batch 14, batch-loss: None
step15: epoch 1, batch 15, batch-loss: None
step16: epoch 2, batch 1, batch-loss: None
step17: epoch 2, batch 2, batch-loss: None
step18: epoch 2, batch 3, batch-loss: None
step19: epoch 2, batch 4, batch-loss: None
step20: epoch 2, batch 5, batch-loss: None
step21: epoch 2, batch 6, batch-loss: 